In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os
from PIL import Image
from tqdm import tqdm

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Set up the path to your image folder in Google Drive
IMAGE_FOLDER = '/content/drive/MyDrive/Path-to-the-data'

In [4]:
# Image parameters
IMG_HEIGHT = 32
IMG_WIDTH = 32
CHANNELS = 3

# Model parameters
LATENT_DIM = 100
BATCH_SIZE = 64

In [ ]:
# Load and preprocess images
def load_images(folder_path):
    images = []
    for filename in tqdm(os.listdir(folder_path)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).resize((IMG_WIDTH, IMG_HEIGHT))
            img_array = np.array(img) / 127.5 - 1  # Normalize to [-1, 1]
            images.append(img_array)
    return np.array(images)

# Load images
images = load_images(IMAGE_FOLDER)
print(f"Loaded {len(images)} images.")

In [6]:
# Create tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(len(images)).batch(BATCH_SIZE)

In [7]:
def make_generator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4*4*256, use_bias=False, input_shape=(LATENT_DIM,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Reshape((4, 4, 256)),

        tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Conv2DTranspose(CHANNELS, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

In [8]:
# Discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[IMG_HEIGHT, IMG_WIDTH, CHANNELS]),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])
    return model

In [ ]:
# Create models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Define loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [10]:
# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [11]:
# Training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        # Print losses every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}")

        # Generate and save images every 50 epochs
        if (epoch + 1) % 50 == 0:
            generate_and_save_images(generator, epoch + 1)


In [ ]:
# Generate and save images
def generate_and_save_images(model, epoch):
    test_input = tf.random.normal([16, LATENT_DIM])
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, :] * 0.5 + 0.5))  # Scale back to [0, 1]
        plt.axis('off')

    plt.savefig(f'/content/drive/MyDrive/gan_output/image_at_epoch_{epoch:04d}.png')
    plt.close()

# Create output directory
os.makedirs('/content/drive/MyDrive/gan_output', exist_ok=True)

# Train the model
EPOCHS = 3000 # This number can be optional
train(dataset, EPOCHS)